In [ ]:
import pandas as pd
import numpy as np
import sklearn

In [ ]:
tr_data = pd.read_parquet('train.parquet')
test_data = pd.read_parquet('test.parquet')
sample = pd.read_csv('sample_submission.csv')

In [ ]:
sample.head(5)

,id,score
0,6125.0,0.757097
1,26781.0,0.346173
2,13333.0,0.431305
3,53218.0,0.847472
4,84204.0,0.065298


In [ ]:
#Для анализа временного ряда выделяем его статистические признаки

def extract_features(data):
    features = pd.DataFrame()
    features['id'] = data['id']

    # Рассчитываем статистические признаки
    features['mean'] = data['values'].apply(np.mean)
    features['std'] = data['values'].apply(np.std)
    features['min'] = data['values'].apply(np.min)
    features['max'] = data['values'].apply(np.max)
    features['median'] = data['values'].apply(np.median)

    # Временные признаки
    features['length'] = data['values'].apply(len)
    features['trend'] = data['values'].apply(lambda x: np.polyfit(range(len(x)), x, 1)[0])

    # Сезонность: отклонение от тренда (остаются циклические колебания)
    def seasonality_score(series):
        detrended = series - np.polyval(np.polyfit(range(len(series)), series, 1), range(len(series)))
        return np.std(detrended)
    features['seasonality'] = data['values'].apply(seasonality_score)

    # Периодичность: автокорреляция на лаге 1 для выявления краткосрочной периодичности
    def autocorrelation(series, lag=1):
        return pd.Series(series).autocorr(lag=lag)
    features['periodicity1'] = data['values'].apply(lambda x: autocorrelation(x, lag=1))

    if 'label' in data.columns:
      features['label'] = data['label']

    return features

def load_and_process_data(file_path):
    data = pd.read_parquet(file_path)
    features = extract_features(data)
    return features

# Для обучающей выборки
train_data = load_and_process_data('train.parquet')
train_data.to_csv('train_features.csv', index=False)

# Для тестовой выборки
test_data = load_and_process_data('test.parquet')
test_data.to_csv('test_features.csv', index=False)

In [ ]:
# Воспользуемся моделью случайного леса

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import joblib

# Загрузка данных и разделение на признаки и целевую переменную
train_data = pd.read_csv('train_features.csv')
X = train_data.drop(['id', 'label'], axis=1)
y = train_data['label']

# Разделение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Оценка качества модели
y_pred_proba = model.predict_proba(X_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f"ROC AUC на валидационной выборке: {roc_auc:.4f}")

# Сохранение модели
joblib.dump(model, 'final_model.pkl')

ROC AUC на валидационной выборке: 0.8735


['final_model.pkl']

In [ ]:
test_data = pd.read_csv('test_features.csv')
submission = pd.DataFrame()
submission['id'] = test_data['id']

X = test_data.drop(['id'], axis=1)
submission['score'] = model.predict_proba(X)[:, 1]
submission['id'] = submission['id'].astype(float)
submission.to_csv('submission.csv', index=False)